In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 65.9 MB/s 
     |████████████████████████████████| 7.6 MB 54.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.8 MB/s 
     |████████████████████████████████| 212 kB 63.3 MB/s 
     |████████████████████████████████| 115 kB 65.2 MB/s 
     |████████████████████████████████| 127 kB 65.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
foldername= "/content/drive/My Drive/nlpproject/"

In [ ]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

In [ ]:
class CFG:
    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    basic_lr=1e-3
    train = True
    debug = False
    offline = False
    models_path = "bert-base-uncased"
    epochs = 50
    save_all_models = False
    apex = True
    print_freq = 20
    num_workers = 4
    model = "bert-base-uncased"
    loss_func = 'SmoothL1'
    scheduler = 'cosine'
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    llrd = True
    layerwise_lr = 5e-5
    layerwise_lr_decay = 0.9
    layerwise_weight_decay = 0.01
    layerwise_adam_epsilon = 1e-6
    layerwise_use_bertadam = False
    #pooling
    pooling = 'mean' # mean, max, min, attention, weightedlayer
    layer_start = 4
    #init_weight
    init_weight = 'normal' # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    #re-init
    reinit = True
    reinit_n = 1
    #adversarial
    fgm = False
    awp = False
    adv_lr = 1
    adv_eps = 0.2
    unscale = False
    eps = 1e-6
    betas = (0.9, 0.999)
    max_len = 100
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    target_cols = ['EI', 'SN', 'TF', 'JP']
    seed = 42
    cv_seed = 42
    n_fold = 4
    trn_fold = list(range(n_fold))
    batch_size = 2048
    n_targets = 4
    gpu_id = 0
    device = f'cuda:{gpu_id}'
cfg=CFG()

In [ ]:

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained(cfg.model)
#the dataset class for the first dataset, tokenized, and labeled
class Ds_EI(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=0  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1
        else:
            return 0
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]

class Ds_SN(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=1  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1.
        else:
            return 0.
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]

class Ds_TF(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=2  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1.
        else:
            return 0.
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]

class Ds_JP(Dataset):
    def __init__(self, path, tokenizer, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=3  #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        type=item["type"]
        labels=self.str2label(type)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1
        else:
            return 0
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]



Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
path=foldername+"dataset2.csv"
#print(dataset[0])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def getdl(ds):
    total_len=len(ds)
    train_len=int(len(ds)*0.9)
    val_len=int((total_len-train_len)/2)
    test_len=total_len-train_len-val_len
    [train_ds, val_ds, test_ds]=torch.utils.data.random_split(ds, [train_len, val_len, test_len])
    #return (training dataloader, validation dataloader, test dataloader)
    return len(train_ds), len(val_ds), len(test_ds), DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)
    #return DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)

In [ ]:
#NEED TO CHANGE WHEN SWITCH TASK
ds=Ds_EI(path, tokenizer)
len_train, len_val, len_test, train_dl_EI,val_dl_EI, test_dl_EI=getdl(ds)

In [ ]:
print(len(ds))

396523


In [ ]:
for d in ds:
  

In [ ]:
def evaluate(labels, outputs):
  answers=(torch.argmax(outputs, dim=1))
  allcorrect=torch.sum(answers==labels)
  return allcorrect

In [ ]:
class Model(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim

        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.fc1= nn.Linear(hidden_dim, 128)
        self.fc2= nn.Linear(128, 32)
        self.fc3= nn.Linear(32, 8)
        self.fc_final= nn.Linear(8, 2)
    def forward(self, input_ids):
        o =  self.word_embeddings(input_ids)
        o,_= self.lstm(o)
        o=o[:,-1]
        o = self.fc1(o)
        o = self.fc2(o)
        o = self.fc3(o)
        o = self.fc_final(o)
        return o

In [ ]:
from torch.optim import lr_scheduler
from torch.nn import CrossEntropyLoss
from torch import optim
from torch.optim import Adam
from tqdm.notebook import tqdm
def train(train_ds, eval_ds, model, epochs, cfg, type, lr, loss=None):
    if torch.cuda.is_available():  
        dev = "cuda:0" 
    else:  
        dev = "cpu" 
    device = torch.device(dev)
    model = model.to(device)

    #weights=torch.tensor([1., 3.]).cuda()
    #criterion = nn.MSELoss()
    criterion = CrossEntropyLoss(weight=LABEL_RATIO)

    criterion.to(device)
    #criterion = loss
    
    optimizer = Adam(model.parameters(), lr=lr)
    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=500, eta_min=1e-15)
    #scheduler= optim.lr_scheduler.ExponentialLR(optimizer, 0.99, last_epoch=- 1, verbose=False)
        
    totalevalloss=0
    totalcorrect=0
    totaldata=0
    with torch.no_grad():
        model.eval()
        for batch in eval_ds:
            batch.to(device)
            blabels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            eloss=criterion(outputs, blabels).item()
            totalevalloss+=eloss
            totalcorrect+=evaluate(blabels, outputs)
            totaldata+=len(blabels)
    totalcorrect_rate=(totalcorrect/(totaldata))
    print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
    #print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} ' ) 
    print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} | current lr: {scheduler.get_last_lr()}' ) 
    
    for e in range(epochs):
        totaltrainloss=0
        totaltraincorrect=0
        totaltraindata=0
        for i,batch in enumerate(train_ds):
            model.train()
            optimizer.zero_grad()
            batch.to(device)
            labels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            bloss=criterion(outputs, labels)
            bloss.backward()
            optimizer.step()
            totaltrainloss+=bloss.item()
            totaltraincorrect+=evaluate(labels, outputs)
            totaltraindata+=len(labels)
        scheduler.step()
        totalevalloss=0
        totalcorrect=0
        totaldata=0
        with torch.no_grad():
            model.eval()
            for batch in eval_ds:
                batch.to(device)
                blabels=batch["labels"]
                outputs=model(input_ids=batch["input_ids"])
                eloss=criterion(outputs, blabels).item()
                totalevalloss+=eloss
                totalcorrect+=evaluate(blabels, outputs)
                totaldata+=len(blabels)
        totalcorrect_rate=(totalcorrect/(totaldata))
        totaltraincorrect_rate = (totaltraincorrect/(totaltraindata))
        print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
        print("probability that our prediction of ", type, " is correct in training dataset: ", totaltraincorrect_rate)
        #print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .3f}' ) 
        print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .8f} | current lr: {scheduler.get_last_lr()}' ) 

In [ ]:
model=Model(cfg, 0.2)
train(train_dl_EI, val_dl_EI, model, epochs=cfg.epochs, cfg=cfg, type="EI", lr=1e-3, loss=None)

TypeError: ignored

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 
device = torch.device(dev)
cpu=torch.device("cpu")
features=[]
labels=[]
model.to(device)
with torch.no_grad():
    for i,batch in enumerate(dl_EI):
        print(i*cfg.batch_size)
        batch.to(device)
        #last_hidden_layers=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"]).last_hidden_state[:,0]
        #last_hidden_layers.to("cpu")
        #features.extend(last_hidden_layers)
        output=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"])
        #print(output)
        output=output.last_hidden_state[:,0].cpu()
        #print(pooler_output[0])
        features.extend(output)
        l=batch["labels"].to("cpu")
        labels.extend(l)